<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/03_langchain/02_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers.

In [ ]:
# Set the model variable based on the best and cheapest available choice at the current date
llm_model = "gpt-4o-mini"

In [ ]:
#!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain langchain-openai langchain-community
# langchain: core framework (chains, prompts, base classes)
# langchain-openai: OpenAI models (ChatOpenAI, embeddings)
# langchain-community: community integrations (memory, message history)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
# Create a reusable chat prompt template with a {product} placeholder (can be reused by inserting different product names).
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [ ]:
# Create a chain that links the LLM with the prompt template to generate responses (use of RunnableSequence as LLMChain as a replacement for LLMChain)
from langchain_core.runnables.base import RunnableSequence
chain = RunnableSequence(prompt, llm)

In [ ]:
product = "Queen Size Sheet Set"
response = chain.invoke(product)
print(response)

content="Choosing a name for a company that specializes in queen size sheet sets can be both fun and strategic. Here are some suggestions that convey comfort, quality, and a focus on queen-sized bedding:\n\n1. **Queen's Comfort**\n2. **Majestic Sheets**\n3. **Regal Rest**\n4. **Dreamy Queen**\n5. **Sovereign Sheets**\n6. **Royal Slumber**\n7. **Crown Comforts**\n8. **Queen Size Dreams**\n9. **Luxuria Linens**\n10. **Serene Queen Bedding**\n\nMake sure to check for the availability of these names in terms of domain names and trademarks before making a final decision!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 23, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'i

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain # (single input/outout)

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name (only one) to describe a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"QueenDreams"
QueenDreams: Empowering women with luxurious sleepwear and accessories, blending comfort and elegance for a royal night's rest. Embrace your dreams!

> Finished chain.


"QueenDreams: Empowering women with luxurious sleepwear and accessories, blending comfort and elegance for a royal night's rest. Embrace your dreams!"

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain # (multiple inputs/outputs)

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output=summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input=summary, language and output=followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: input= Review and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, which is strange. I buy the same ones in stores and the taste is much better...  \nOld batch or counterfeit!?",
 'summary': 'The reviewer finds the taste mediocre and the foam ineffective, suggesting the product may be an old batch or counterfeit compared to better versions purchased in stores.',
 'followup_message': "Je vous remercie pour votre retour d'expérience. Nous sommes désolés d'apprendre que le produit n'a pas répondu à vos attentes en termes de goût et de mousse. Votre remarque sur la possibilité d'un lot ancien ou d'une contrefaçon est très importante pour nous. Pouvez-vous nous donner plus de détails sur votre achat ? Cela nous aidera à vérifier la qualité et à améliorer nos produits. Nous nous engageons à vous fournir la meilleu

## Router Chain

In [ ]:
# Define specialized prompt templates for different subjects with {input} as the placeholder for questions
physics_template = """You are a very smart physics professor. You are great at answering questions about physics in a concise and easy to understand manner. When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. You have an excellent knowledge of and understanding of people, events and contexts from a range of historical periods. You have the ability to think, reflect, debate, discuss and evaluate the past. You have a respect for historical evidence and the ability to make use of it to support your explanations and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist. You have a passion for creativity, collaboration, forward-thinking, confidence, strong problem-solving capabilities, understanding of theories and algorithms, and excellent communication skills. You are great at answering coding questions. You are so good because you know how to solve a problem by describing the solution in imperative steps that a machine can easily interpret and you know how to choose a solution that has a good balance between time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
# Organize subject-specific prompts into a list of dictionaries with name, description, and template
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:
# Create LLM chains for each subject and store them in a dictionary; also prepare a string listing all destinations (destinations_str)
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
# destinations
# destinations_str

In [ ]:
# Create a prompt template and wrap it in an LLM chain to generate responses
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
# Template for a multi-prompt router that selects the most suitable prompt for a given input
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of the candidate prompts listed below. If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
# Fill the multi-prompt router template with available destinations, create a PromptTemplate for it, and wrap it in an LLMRouterChain to automatically choose the best subject-specific chain for a given input
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
# Combine the router and all subject-specific chains into a MultiPromptChain that routes inputs to the best chain
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body," which absorbs all incident radiation, regardless of frequency or angle. A perfect black body is a theoretical concept that does not reflect or transmit any light, making it appear completely black at room temperature.\n\nThe key characteristics of black body radiation include:\n\n1. **Temperature Dependence**: The spectrum of radiation emitted by a black body depends solely on its temperature. As the temperature increases, the intensity of radiation increases and the peak wavelength shifts to shorter wavelengths, a phenomenon described by Wien\'s displacement law.\n\n2. **Planck\'s Law**: Max Planck formulated a law that describes the spectral distribution of black body radiation. It shows that the energy emitted at a given wavelength is quantized, leading to the concept of quantized energy levels in quantum mechanics.\n\n3. **Stefan-Boltzmann Law**: This law states tha

In [ ]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve the problem \\(2 + 2\\), we can break it down into its component parts:\n\n1. Identify the numbers involved: We have the numbers 2 and 2.\n2. Understand the operation: The operation we are performing is addition.\n\nNow, we can combine the two numbers:\n\n\\[\n2 + 2 = 4\n\\]\n\nSo, the answer to the question \\(2 + 2\\) is \\(4\\).'

In [ ]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the genetic blueprint for the organism. Here are several key reasons why this is the case:\n\n1. **Genetic Information**: DNA contains the instructions needed for the development, functioning, growth, and reproduction of all living organisms. It encodes the information necessary to produce proteins, which perform a vast array of functions in the body.\n\n2. **Cellular Function**: Each cell type in the body has specific functions, and the DNA in each cell contains the genes that are necessary for that cell's role. For example, muscle cells have genes that help them contract, while nerve cells have genes that support their ability to transmit signals.\n\n3. **Development and Differentiation**: During the development of an organism, all cells originate from a single fertilized egg, which contains DNA. As the organism grows, cells divide and differentiate into various types, but they all retain the same DNA. This ensures that all c

## **Overview of Chain Types in LangChain**

* **SimpleSequentialChain**  
  Runs multiple steps one after another, passing the output of one step as the input to the next.

  Example: summarize a text, then translate the summary. Best for straight, linear workflows.

* **SequentialChain**  
  Similar to SimpleSequentialChain but more flexible. Allows multiple inputs and outputs per step and controls how data flows between steps.

  Example: extract key facts from text, then generate questions from those facts. Useful for slightly more complex workflows.

* **RouterChain**  
  Routes inputs to the appropriate chain based on content rather than running steps sequentially.

  Example: if you have chains for math, history, and physics questions, RouterChain selects the correct chain automatically. Ideal for handling multiple types of tasks intelligently.